In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

csv = 'clean_tweet.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)

x = my_df.text
y = my_df.target


/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
from sklearn.cross_validation import train_test_split

x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=1500)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=1500)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils

def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  if sys.path[0] == '':


In [6]:
all_x

327222           wishing could get smoothie from jamba juice
746985                            it here ickgrossdisgusting
738670             about to finally knuckle down to revision
420170     and let me finally be happy im just so over ev...
1571286             is off up pendle hill with college today
218805     would suggest switching to sony vegas for the ...
415446                           it finally hit me and upset
217185               that because it cool wish had telescope
422884     pissed off as still can not contact oasisinet ...
1587416    hehe that cool well doin sth with friends swim...
792874           still unsure of this affair ilh neverchange
149668     oohh nice was just wondering though the missus...
760939     you ladies enjoy puerto rico for me can not be...
1159769    via hey there great to meet you my site is whe...
1409327    let get that win boys ll be cheering you on al...
848458     jealous we are heading towards winter here in ...
462237     its awesome w

In [7]:
all_x_w2v

[LabeledSentence(words=['wishing', 'could', 'get', 'smoothie', 'from', 'jamba', 'juice'], tags=['all_327222']),
 LabeledSentence(words=['it', 'here', 'ickgrossdisgusting'], tags=['all_746985']),
 LabeledSentence(words=['about', 'to', 'finally', 'knuckle', 'down', 'to', 'revision'], tags=['all_738670']),
 LabeledSentence(words=['and', 'let', 'me', 'finally', 'be', 'happy', 'im', 'just', 'so', 'over', 'everything'], tags=['all_420170']),
 LabeledSentence(words=['is', 'off', 'up', 'pendle', 'hill', 'with', 'college', 'today'], tags=['all_1571286']),
 LabeledSentence(words=['would', 'suggest', 'switching', 'to', 'sony', 'vegas', 'for', 'the', 'trip', 'but', 'it', 'does', 'not', 'work', 'on', 'mac', 'apparently'], tags=['all_218805']),
 LabeledSentence(words=['it', 'finally', 'hit', 'me', 'and', 'upset'], tags=['all_415446']),
 LabeledSentence(words=['that', 'because', 'it', 'cool', 'wish', 'had', 'telescope'], tags=['all_217185']),
 LabeledSentence(words=['pissed', 'off', 'as', 'still', 'c

In [22]:
cores = multiprocessing.cpu_count()

model_ug_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

/usr/local/lib/python3.5/dist-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 1596522/1596522 [00:00<00:00, 3292297.13it/s]


In [23]:
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha


100%|██████████| 1596522/1596522 [00:00<00:00, 3162605.56it/s]


In [4]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

In [12]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)

In [5]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)

    

NameError: name 'train_vecs_dbow' is not defined

In [16]:
print(clf.score(validation_vecs_dbow, y_validation))
model_ug_dbow.save('d2v_model_ug_dbow.doc2vec')
model_ug_dbow = Doc2Vec.load('d2v_model_ug_dbow.doc2vec')
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


0.7409332915753211


In [6]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])



/usr/local/lib/python3.5/dist-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 1596522/1596522 [00:00<00:00, 3118129.56it/s]


In [7]:
for epoch in range(30):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

100%|██████████| 1596522/1596522 [00:00<00:00, 3352282.67it/s]


In [8]:
model_ug_dmc.save('d2v_model_dmc.doc2vec')
print("Saved the dmc model")

Saved the dmc model


In [13]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 100)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
print(clf.score(validation_vecs_dmc, y_validation))


0.6639523958659568


In [16]:
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])



/usr/local/lib/python3.5/dist-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 1596522/1596522 [00:37<00:00, 42484.75it/s]


In [17]:
for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha


100%|██████████| 1596522/1596522 [00:00<00:00, 3413114.53it/s]


In [18]:
model_ug_dmm.save('d2v_model_dmm.doc2vec')
print("Saved the dmm model")

Saved the dmm model


In [20]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [24]:
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_validation, 200)


In [25]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
clf.score(validation_vecs_dbow_dmm, y_validation)


0.7542749765111181